In [61]:
import pandas as pd
import os
import cv2

In [62]:
files=os.listdir('wbf_5_fold/')

In [63]:
files

['subs_fold_1.csv',
 'subs_fold_3.csv',
 'subs_fold_2.csv',
 'subs_fold_5.csv',
 'subs_fold_4.csv']

In [64]:
dfs = [pd.read_csv(f'wbf_5_fold/{i}') for i in files ]

In [65]:
test=os.listdir('test/images/')
test = [i for i in test if ".jpg" in i]

In [66]:
box_col=['XMin', 'YMin', 'XMax', 'YMax']

In [67]:
final_df = pd.DataFrame()

In [68]:
weights=[1]*len(files)

In [69]:
iou_thr = 0.5
skip_box_thr = 0.0001
sigma = 0.1

In [70]:
for name in test:
    h,w,c = cv2.imread(f'test/images/{name}').shape
    box_list=[]
    score_list =[]
    label_list= []
    for df in dfs:
        temp = df.loc[df.ImageID==name]
        temp[['XMin','XMax']] = temp[['XMin','XMax']]/w
        temp[['YMin','YMax']] = temp[['YMin','YMax']]/h
        list_box = list([list(i) for i in temp[box_col].values])
        box_list.append(list_box)
        score_list.append(list(temp.Conf.values))
        label_list.append([1]*len(list(temp.Conf.values)))
    boxes, scores, labels = weighted_boxes_fusion(box_list, 
                                              score_list, 
                                              label_list, 
                                              weights=weights, 
                                              iou_thr=iou_thr, 
                                              skip_box_thr=skip_box_thr)
    box_ = pd.DataFrame(boxes,columns=box_col)
    box_[['XMin','XMax']] = box_[['XMin','XMax']]*w
    box_[['YMin','YMax']] = box_[['YMin','YMax']]*h
    box_['Conf'] = scores
    box_['ImageID'] = name
    box_['LabelName'] = 'pothole'
    final_df = pd.concat([final_df,box_])

In [71]:
final_df.shape

(8303, 7)

In [72]:
final_df.to_csv('wbf_final.csv',index=None)

In [73]:
box_

,XMin,YMin,XMax,YMax,Conf,ImageID,LabelName
0,85.356190,277.453108,255.736095,398.620342,0.450133,806.jpg,pothole
1,116.405045,328.812389,239.720097,376.711510,0.082478,806.jpg,pothole
2,117.772702,270.809212,249.157409,349.850555,0.029568,806.jpg,pothole
3,0.000000,255.172379,248.061127,442.541840,0.009640,806.jpg,pothole
4,466.242676,327.908630,502.434265,395.974640,0.007489,806.jpg,pothole
5,446.065002,332.841675,503.000000,421.546783,0.003275,806.jpg,pothole
6,62.468082,307.469147,207.643555,409.456360,0.002580,806.jpg,pothole
7,402.526031,33.305428,468.053040,53.999214,0.002573,806.jpg,pothole


In [74]:
temp

,XMin,YMin,XMax,YMax,Conf,LabelName,ImageID
1357,0.158872,0.475769,0.502125,0.687662,0.734886,pothole,806.jpg
1358,0.235071,0.580530,0.478237,0.657168,0.159031,pothole,806.jpg
1359,0.223450,0.479750,0.496726,0.610305,0.052697,pothole,806.jpg


In [44]:
from ensemble_boxes import *


In [45]:
boxes

array([[ 87.22399248, 277.98235703, 257.94321603, 395.89312363],
       [116.65795058, 329.82907104, 234.51495174, 373.98422241],
       [120.75105026, 267.74806023, 248.77213633, 348.91726685],
       [  0.        , 255.17237854, 248.06112671, 442.5418396 ],
       [466.24267578, 327.90863037, 502.43426514, 395.97463989],
       [ 62.46808243, 307.46914673, 207.64355469, 409.45635986],
       [402.52603149,  33.30542755, 468.05303955,  53.99921417]])

In [46]:
df.loc[df.ImageID==name]

,XMin,YMin,XMax,YMax,Conf,LabelName,ImageID
1068,88.177574,270.657257,246.616638,398.769958,0.341014,pothole,806.jpg
1069,116.721703,329.708954,233.669357,374.056427,0.107364,pothole,806.jpg
1070,118.230301,274.222931,251.048248,351.136810,0.030413,pothole,806.jpg
